In [ ]:
import pandas as pd
import textwrap

from dotenv import load_dotenv
import pandas as pd
import os
from pinecone import Pinecone, ServerlessSpec
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_pinecone import PineconeVectorStore
from langchain_openai import OpenAIEmbeddings
from tqdm import tqdm

### 전역변수 가져와서 넣기

In [2]:
table = pd.read_csv("/home/yhkim/code/JobPT/backend/get_similarity/data/korean_jd_105.csv")

In [3]:
##기존에는 평문이라 retrieval에 큰 문제는 없었는데 이런 데이터는 일정구분자로 구분해서 평문으로 만들어주는게 좋을듯 하다
##물론 최종적으로는 본문과 같은 형태를 만드는게 추천에 베스트로 생각됨
table.head()

,url,job_category,job_name,title,company_name_raw,company_name,location,experience_requirement,company_id,tag_name,tag_id,description,main_work,qualification,preferences,welfare,tech_list
0,https://www.wanted.co.kr/wd/267127,10110,소프트웨어 엔지니어,[라이브클래스] Back-end Engineer,퓨쳐스콜레∙서울 성동구∙경력 5년 이상,퓨쳐스콜레,서울 성동구,경력 5년 이상,/company/18143,NaN,NaN,Mission : 우리는 지식으로 자신과 세상의 변화를 꿈꾸는 사람들을 연결합니다....,주요업무 퓨쳐스콜레 프로덕트팀에서 LiveKlass 서비스의 성장과 기술적인 혁신을...,자격요건 다음 기술과 경험을 보유하신 분을 찾습니다. • 5년 이상의 Back-en...,우대사항 다음 경험이 있으시면 더욱 좋습니다. • 커머스 플랫폼을 경험하신 분 • ...,"혜택 및 복지 • 성과에 대해서도 적극적으로 보상해요! 성과급, 스톡옵션 지급 • ...",NaN
1,https://www.wanted.co.kr/wd/265175,10110,소프트웨어 엔지니어,AI 소프트웨어 풀스택 개발자 (3~12년),로이드케이∙경기 성남시∙경력 3-10년,로이드케이,경기 성남시,경력 3-10년,/company/36974,NaN,NaN,로이드케이팀은 기술로 상상을 현실로 만드는 사람들입니다. 데이터·AI·클라우드·보안...,"주요업무 AI 소프트웨어 풀스택 개발자로서, 다음과 같은 업무를 수행합니다. • L...","자격요건 • Al/ML 및 딥러닝 기술에 대한 이해 • MCP, A2A 등의 Pro...",우대사항 • 기술적 고민/성취에 대해 공유하고 팀웍을 통해 해결할 수 있는 커뮤니케...,혜택 및 복지 • 업무 몰입을 위한 최신 장비는 회사가 책임집니다. :) • 필요한...,NaN
2,https://www.wanted.co.kr/wd/315226,10110,소프트웨어 엔지니어,BMS 소프트웨어 엔지니어,한화솔루션큐셀부문(한화큐셀)∙서울 금천구∙경력 3-13년,한화솔루션큐셀부문(한화큐셀),서울 금천구,경력 3-13년,/company/19159,NaN,NaN,"큐셀 부문은 글로벌 태양광 Total Solution 기업으로, 태양광 셀, 모듈 ...",주요업무 • 공통 - BMS 시스템 요구사항 기반 소프트웨어 설계 및 구현 - AU...,"자격요건 [필수요건] • 전자전기, 제어, 컴퓨터, 소프트웨어 등 유관 전공 학사 ...",우대사항 • AUTOSAR 기반 BSW/ASW 개발 경험 • ISO 26262 기능...,혜택 및 복지 복리후생/교육 등 자사 홈페이지 참고,NaN
3,https://www.wanted.co.kr/wd/265172,10110,소프트웨어 엔지니어,빅데이터 솔루션 데이터 엔지니어 (~10년 미만),로이드케이∙경기 성남시∙신입-경력 8년,로이드케이,경기 성남시,신입-경력 8년,/company/36974,NaN,NaN,로이드케이는 데이터·AI·보안·클라우드 분야에서 기업의 디지털 전환을 이끄는 기술 ...,주요업무 ㆍ대용량/실시간 데이터 수집/처리 및 모니터링 시스템 설계/운영 ㆍ키워드/...,자격요건 ㆍPython/Shell Script 기반 데이터 파이프라인 자동화 경험 ...,"우대사항 ㆍ컴퓨터 공학, 시스템 공학 등 관련 전공 ㆍ대용량/실시간 데이터 수집 파...",혜택 및 복지 • 업무 몰입을 위한 최신 장비는 회사가 책임집니다. :) • 필요한...,NaN
4,https://www.wanted.co.kr/wd/314976,10110,소프트웨어 엔지니어,iOS 신입 개발자,딥메디∙서울 관악구∙신입,딥메디,서울 관악구,신입,/company/10409,NaN,NaN,딥메디(Deepmedi)는 카메라 기반 디지털 바이오마커 분석 기술을 통해 보다 쉽...,주요업무 ○ UIKit 기반 애플리케이션 개발을 담당 ○ 배포앱 리팩토링 및 유지보...,자격요건 ○ 신입 또는 경력 1년이하 ○ UIKit을 활용한 전반적인 경험과 Sna...,우대사항 ○ MVVM 아키텍처 패턴에 대한 이해 및 실제 프로젝트 적용 경험 ○ 카...,"혜택 및 복지 ○ 주 52시간제 준수, 자율 근무제 (오전 8 ~ 10시 출근, 오...",NaN


In [52]:
print(len(table))
print(len(table.drop_duplicates()))

105
105


In [4]:
table.columns

Index(['url', 'job_category', 'job_name', 'title', 'company_name_raw',
       'company_name', 'location', 'experience_requirement', 'company_id',
       'tag_name', 'tag_id', 'description', 'main_work', 'qualification',
       'preferences', 'welfare', 'tech_list'],
      dtype='object')

In [6]:
temp = table.iloc[0]
print(temp)

url                                      https://www.wanted.co.kr/wd/267127
job_category                                                          10110
job_name                                                         소프트웨어 엔지니어
title                                            [라이브클래스] Back-end Engineer
company_name_raw                                      퓨쳐스콜레∙서울 성동구∙경력 5년 이상
company_name                                                          퓨쳐스콜레
location                                                             서울 성동구
experience_requirement                                             경력 5년 이상
company_id                                                   /company/18143
tag_name                                                                NaN
tag_id                                                                  NaN
description               Mission : 우리는 지식으로 자신과 세상의 변화를 꿈꾸는 사람들을 연결합니다....
main_work                 주요업무 퓨쳐스콜레 프로덕트팀에서 LiveKlass 서비스의 성장과 기술적인 혁신을...
qualificatio

### 데이터 처리

In [40]:
###청킹 전 메타데이터도 embedding 영역에추가 아래 보이는 3개 영역을 나누는 구분자 추가

sentence = "<chunk_sep>".join([temp["description"],\
f"\n\n하는일: {temp['main_work']}\n\n자격요건: {temp['qualification']}",\
f"\n\nurl: {temp['url']}\n\njob_name: {temp['job_name']}\n\ncompany_name: {temp['company_name']}\n\nwelfare: {temp['welfare']}"])

In [41]:
print(sentence)

Mission : 우리는 지식으로 자신과 세상의 변화를 꿈꾸는 사람들을 연결합니다. Vision : 사는 곳이 배움의 기회를 제한하지 않는 세상을 만듭니다. [지식으로 세상을 연결하는 라이브클래스] 우리는 위 미션을 바탕으로 비전을 이루기 위해 ‘라이브클래스’와 '라이브클래스Pro'를 만들고 있습니다. ‘라이브클래스’는 나만의 온라인 강의 사이트를 손쉽게 만들 수 있는 지식 비즈니스 플랫폼으로 사람들이 자신의 지식을 가장 쉽고 빠르게 판매할 수 있도록 돕습니다. '라이브클래스Pro'는 대규모의 챌린지 또는 코호트 교육을 하는 기업에 최적화된 플랫폼으로 전용 앱과 웹을 제공합니다. [국내 1위 지식 비즈니스 플랫폼, 라이브클래스] • 2만 여개의 고객사, 월 50만명이 사용하는 플랫폼 • 창업 2년 만에 월 BEP 달성, 연 평균 500%의 거래액 성장 기록 • 정부 지자체, 대학, 민간기관 등 100여곳의 기관에서 활용하는 라이브클래스 • 70억 이상의 누적 투자 유치(뮤렉스파트너스, 티비티파트너스, HB인베스트먼트, 임팩트스퀘어 등) • 대한민국 대표 강사 김미경(MKYU)의 '미모리안' 등 각 분야별 1위 기업 및 지식 크리에이터들을 통한 성장 [우리는 이렇게 일하고 성장해요.] 1. 막연한 불안과 걱정 대신, 내가 바꿀 수 있는 것(영향력의 원)에 집중해요. 2. 고객이 다섯을 원할 때, 열을 만족 시키기 위해 노력해요. 3. 빠르게 실행하고, 데이터에 기반해 집요하게 개선해요. 4. 최고의 서비스를 만들기 위해 한계를 두지 않고 지속적으로 성장해요. 5. No라고 말하는 것을 두려워하지 않고, 결국 Yes를 이끌어 내요. 6. 상대방을 존중하되, 대안을 가지고 솔직하게 소통해요. 7. 미친 가속을 원하는 사람도, 워라밸을 원하는 사람도 상호 성장하는 문화를 만들어 가요. 8. 현재 내가 누리는 것에 감사하며, 대접 받고 싶은 대로 먼저 실천해요. 라이브클래스와 세상을 바꾸는 여정에 함께할 당신을 기다립니다. 다양한 백그라운드의 멋진 팀원들과 협업하며 엄

In [47]:
table["sentence"] = table.apply(lambda x: "<chunk_sep>".join([x["description"],\
f"\n\n하는일: {x['main_work']}\n\n자격요건: {x['qualification']}",\
f"\n\nurl: {x['url']}\n\njob_name: {x['job_name']}\n\ncompany_name: {x['company_name']}\n\nwelfare: {x['welfare']}"]), axis=1)

In [48]:
print(table["sentence"][42])

[조직 소개] CJ ENM 엔터테인먼트 부문 Mnet Plus 사업부는 CJ ENM의 글로벌 KPOP 플랫폼인 Mnet Plus의 개발과 사업을 담당하는 조직으로, Mnet Plus는 Global K-Culture 소비자와 CJ ENM의 콘텐츠, 아티스트, 오프라인 콘서트를 이어주는 디지털 접점을 구축하고 있습니다. 방송/디지털의 콘텐츠, KCON/MAMA 와 같은 Global Offline Event, KPOP 아티스트들의 활동은 Mnet Plus를 통해 전세계 K-POP 팬덤에게 새로운 경험을 전하려고 합니다. 특히, K-POP 팬덤들이 문화 콘텐츠 소비자에 머무르는 것이 아니라, 아티스트의 “발탁-성장-지지-성공”의 전 과정을 함께 만들어내고 공유할 수 있도록 하며, Offline-TV 에서 Mobile까지, 한국에서 글로벌로, 경계없는 경험의 확장과 혁신을 만들어가고자 합니다. 현재 250여개국에서 발생하는 트래픽과 KPOP 팬덤의 높은 인게이지먼트를 기반으로 다양한 서비스 및 수익모델을 준비, 시도 중에 있으며 지속적인 Global Business Opportunity 발굴을 진행할 예정입니다. 주요업무 [포지션] Mnet Plus 백엔드 개발자 • 글로벌 팬덤 플랫폼 서비스인 Mnet Plus 개발 • 방송 이벤트, 콘서트, 투표 개발 • 광고 수익 개선을 위한 기능 개발 [이력서 작성 Tip] • Mnet Plus가 사용하는  Java 또는 Kotlin 경험을 중심으로 지원서를 작성해주세요. 자격요건 [필수 자격] • Java 또는 Kotlin을 사용한 서버 개발 경험 3년 이상 • RDBMS 또는 Nosql DB를 활용한 설계/운영 경험을 보유한 분 • Git 사용, 코드 리뷰에 익숙한 분 우대사항 [우대사항] • 대규모 트래픽을 고려한 시스템 디자인, 개발 경험을 보유한 분 • AWS 및 쿠버네티스 환경에서 서비스 개발/운영 경험을 보유한 분 • 초기 프로젝트에 참여하여 개발 환경 세팅, 아키텍처 수립 경험을 보유한 분 [캐릭터 & 성향] • 

### vectorDB 로드

In [54]:
collection = "korea-jd-test"

index_name = collection
if pc.has_index(index_name):
    print("인덱스가 이미 존재합니다")
    index = pc.Index(index_name)
    ## 데이터가 남아있을때 데이터 제거
    # if len(index.describe_index_stats()["namespaces"]) > 0:
    #     index.delete(delete_all=True, namespace="")

인덱스가 이미 존재합니다


### 청킹&메타데이터 추출

In [57]:
sentences = table["sentence"]
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1024, chunk_overlap=100, separators=["<chunk_sep>","\r\n","\n\n", "\n", "\t", " ", ""])


In [96]:
### 청킹&구분자 제거
total_chunks = []
for i, desciption in enumerate(sentences):
    meta_data = [{"description":table.iloc[i]["description"],
    "job_url": table.iloc[i]["url"],
    "company": table.iloc[i]["company_name"],
    "location": table.iloc[i]["location"],
    "experience_requirement": table.iloc[i]["experience_requirement"],
    
    }]
    chunks = text_splitter.create_documents([desciption], meta_data)

    for chunk in chunks:
        chunk.page_content = chunk.page_content.replace("<chunk_sep>","")
    total_chunks.extend(chunks)
# for k, v in total_chunks[0].metadata.items():
#     ###description은 전문을 보여주기 위해 추가한 메타데이터입니다
#     print(k, v)

### 데이터 업로드

In [103]:
emb_model = OpenAIEmbeddings()
vector_store = PineconeVectorStore(index=index, embedding=emb_model)

total = len(total_chunks)
batch_size = 100

In [104]:
for start in tqdm(range(0, total, batch_size), desc="Upserting to Pinecone"):
    end = start + batch_size
    batch_docs = total_chunks[start:end]
    vector_store.add_documents(documents=batch_docs)

Upserting to Pinecone: 100%|██████████| 6/6 [00:23<00:00,  3.96s/it]


In [ ]:
## 인덱스 생성 확인
try:
    print("DB 상태 확인 중...")
    print(f"- 인덱스 정보: {index.describe_index_stats()}")
except Exception as e:
    print(f"DB 상태 확인 중 에러 발생: {str(e)}")
    raise